In [76]:
import angr, monkeyhex

proj = angr.Project('examples/fauxware/fauxware')
proj.loader

<Loaded fauxware, maps [0x400000:0xc07fff]>

In [88]:
""" 
EL CLE loader representa un conglomerado de objetos binarios cargados y mapeados en un solo espacio de memoria.
Cada objeto binario es cargado por el cargador backend de forma que este puede manejar este tipo de archivo.
SE puede conseguir una lista de todos los objetos que el CLE tiene cargado con loader.all_objects
"""

#Todos los objetos cargados
print(proj.loader.all_objects)

print("//////")

#Este es el objeto "main", este es el unico que se especifica cuando se carga el proyecto.
print(proj.loader.main_object)

print("//////")

#Este es un diccionario armado desde los nombres de los objetos a los objetos. 
print(proj.loader.shared_objects)

print("//////")

#Estos son todos los objetos que estan cargados desde archivos ELF
#Si estamos usando un programa de windows hay que usar "all_pe_objects"
print(proj.loader.all_elf_objects)

print("//////")

#Estos son los objetos externos, se usa para conseguir direcciones de importaciones no resueltas e internas de angr
print(proj.loader.extern_object)

print("//////")

#Este objeto es usado para conseguir direcciones para syscalls emuladas
print(proj.loader.kernel_object)

print("//////")

#Tambien se puede conseguir la referencia a un objeto usando la dirección del mismo.
print(proj.loader.find_object_containing(0x400000))

[<ELF Object fauxware, maps [0x400000:0x60105f]>, <ELF Object libc.so.6, maps [0x700000:0x8f5e77]>, <ELF Object ld-linux-x86-64.so.2, maps [0x900000:0x93730f]>, <ExternObject Object cle##externs, maps [0xa00000:0xa7ffff]>, <ELFTLSObjectV2 Object cle##tls, maps [0xb00000:0xb1500f]>, <KernelObject Object cle##kernel, maps [0xc00000:0xc07fff]>]
//////
<ELF Object fauxware, maps [0x400000:0x60105f]>
//////
OrderedDict({'fauxware': <ELF Object fauxware, maps [0x400000:0x60105f]>, 'libc.so.6': <ELF Object libc.so.6, maps [0x700000:0x8f5e77]>, 'ld-linux-x86-64.so.2': <ELF Object ld-linux-x86-64.so.2, maps [0x900000:0x93730f]>, 'extern-address space': <ExternObject Object cle##externs, maps [0xa00000:0xa7ffff]>, 'cle##tls': <ELFTLSObjectV2 Object cle##tls, maps [0xb00000:0xb1500f]>})
//////
[<ELF Object fauxware, maps [0x400000:0x60105f]>, <ELF Object libc.so.6, maps [0x700000:0x8f5e77]>, <ELF Object ld-linux-x86-64.so.2, maps [0x900000:0x93730f]>]
//////
<ExternObject Object cle##externs, map

In [97]:
"""
Tambien puede extraerse la metadata de estos objetos
"""

obj = proj.loader.main_object

#entry point del objeto
print(obj.entry)

print("@@@@@@@@@@@@")

#Menor y mayor dirección del objeto
print(f"menor: {obj.min_addr} mayor: {obj.max_addr}")

print("@@@@@@@@@@@@")

#Obtiene los segmentos y secciones ELF
print(obj.segments)

print(obj.sections)

print("@@@@@@@@@@@@")

#Se puede obtener un solo segmento o seccion
print(obj.find_segment_containing(obj.entry))
print(obj.find_section_containing(obj.entry))

print("@@@@@@@@@@@@")

#Get the address of the PLT stub for a symbol
addr = obj.plt['strcmp']
print(addr)
obj.reverse_plt[addr]

print("@@@@@@@@@@@@")

#Muestra el objeto base pre enlazado y la ubicacion que esta siendo actualmente mapeada en memoria por CLE
print(obj.linked_base)
print(obj.mapped_base)

4195712
@@@@@@@@@@@@
menor: 4194304 mayor: 6295647
@@@@@@@@@@@@
<Regions: [<ELFSegment flags=0x5, relro=0x0, vaddr=0x400000, memsize=0xa74, filesize=0xa74, offset=0x0>, <ELFSegment flags=0x4, relro=0x1, vaddr=0x600e28, memsize=0x1d8, filesize=0x1d8, offset=0xe28>, <ELFSegment flags=0x6, relro=0x0, vaddr=0x601000, memsize=0x60, filesize=0x50, offset=0x1000>]>
<Regions: [<Unnamed | offset 0x0, vaddr 0x0, size 0x0>, <.interp | offset 0x238, vaddr 0x400238, size 0x1c>, <.note.ABI-tag | offset 0x254, vaddr 0x400254, size 0x20>, <.note.gnu.build-id | offset 0x274, vaddr 0x400274, size 0x24>, <.gnu.hash | offset 0x298, vaddr 0x400298, size 0x1c>, <.dynsym | offset 0x2b8, vaddr 0x4002b8, size 0xd8>, <.dynstr | offset 0x390, vaddr 0x400390, size 0x5a>, <.gnu.version | offset 0x3ea, vaddr 0x4003ea, size 0x12>, <.gnu.version_r | offset 0x400, vaddr 0x400400, size 0x20>, <.rela.dyn | offset 0x420, vaddr 0x400420, size 0x18>, <.rela.plt | offset 0x438, vaddr 0x400438, size 0xa8>, <.init | offset 0x

<h1>Simbolos y realocaciones</h1>

In [98]:
""" 
Tambien se puede trabajar con simbolos usando CLE. Un simbolo es un concepto fundamental en los 
formatos ejecutables, estos mapean nombres a direcciones.

La forma más facil de conseguir un simbolo del CLE es usar loader.find_symbol, que tomara un nombre
o dirección para retornar el objeto.
"""

strcmp = proj.loader.find_symbol('strcmp')
print(strcmp)

<Symbol "strcmp" in libc.so.6 at 0x7a83f0>


In [99]:
""" 
De los atributos más importantes en un simbolo son su nombre, el dueño y la dirección. 
Aunque la dirección puede ser ambigua. Los objetos simbolos tienen tres formas de destacar la misma:
    >.rebased_addr : la dirección en el esacio global de direcciones.
    >.linked_addr : La dirección en el espacio pre-enlazado del binario
    >.relative_addr: Dirección relativa del objeto base. 
"""

print(strcmp.name)
print(strcmp.owner)
print(strcmp.rebased_addr)
print(strcmp.linked_addr)
print(strcmp.relative_addr)

strcmp
<ELF Object libc.so.6, maps [0x700000:0x8f5e77]>
8029168
689136
689136


In [105]:
""" 
Tambien los simbolos soportan la nocion de linkeado dinamico. 
libc provee el simbolo strcmp como export. Y el binario main depende de este. 
Estas cosas pueden ser consultadas
""" 

print(strcmp.is_export)
print(strcmp.is_import)

#On loader, the method is find_symbol because it performs a search operation to find the symbol.
#On a individual object, the method is get_symbol because there can only be one symbol with a given name.

main_strcmp = proj.loader.main_object.get_symbol('strcmp')
print(main_strcmp)
print(main_strcmp.is_export)
print(main_strcmp.is_import)
print(main_strcmp.resolvedby)

True
False
<Symbol "strcmp" in fauxware (import)>
False
True
<Symbol "strcmp" in libc.so.6 at 0x7a83f0>


In [107]:
print(proj.loader.shared_objects['libc.so.6'].imports)

{'_dl_argv': <cle.backends.elf.relocation.amd64.R_X86_64_GLOB_DAT object at 0x7f392030d340>, '__rseq_offset': <cle.backends.elf.relocation.amd64.R_X86_64_GLOB_DAT object at 0x7f392030e630>, '__libc_enable_secure': <cle.backends.elf.relocation.amd64.R_X86_64_GLOB_DAT object at 0x7f392030ef90>, '__libc_stack_end': <cle.backends.elf.relocation.amd64.R_X86_64_GLOB_DAT object at 0x7f392030fda0>, '_rtld_global_ro': <cle.backends.elf.relocation.amd64.R_X86_64_GLOB_DAT object at 0x7f392030e960>, '__rseq_size': <cle.backends.elf.relocation.amd64.R_X86_64_GLOB_DAT object at 0x7f391f88b560>, '_rtld_global': <cle.backends.elf.relocation.amd64.R_X86_64_64 object at 0x7f39229e6b10>, '_dl_find_dso_for_object': <cle.backends.elf.relocation.amd64.R_X86_64_JUMP_SLOT object at 0x7f39229e40b0>, '_dl_deallocate_tls': <cle.backends.elf.relocation.amd64.R_X86_64_JUMP_SLOT object at 0x7f3920206a50>, '__tls_get_addr': <cle.backends.elf.relocation.amd64.R_X86_64_JUMP_SLOT object at 0x7f39229e7fe0>, '_dl_signal_

<h1>Opciones de carga</h1>